In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import eda_functions as eda

In [4]:
import dvc.api

In [5]:
from urllib.parse import urlparse

In [6]:
import warnings 

# Suppressing warning messages
warnings.filterwarnings('ignore')

In [7]:
# set font size 

plt.rcParams.update({'font.size': 12})

# Read The Data

**Files**
- train.csv - historical data including Sales
- test.csv - historical data excluding Sales
- sample_submission.csv - a sample submission file in the correct format
- store.csv - supplemental information about the stores


**Data fields**

Most of the fields are self-explanatory. The following are descriptions for those that aren't.

- **Id** - an Id that represents a (Store, Date) duple within the test set
- **Store** - a unique Id for each store
- **Sales** - the turnover for any given day (this is what you are predicting)
- **Customers** - the number of customers on a given day
- **Open** - an indicator for whether the store was open: 0 = closed, 1 = open
- **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools
- **StoreType** - differentiates between 4 different store models: a, b, c, d
- **Assortment** - describes an assortment level: a = basic, b = extra, c = extended
- **CompetitionDistance** - distance in meters to the nearest competitor store
- **CompetitionOpenSince[Month/Year]** - gives the approximate year and month of the time the nearest competitor was opened (date like)
- **Promo** - indicates whether a store is running a promo on that day
- **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
- **Promo2Since[Year/Week]** - describes the year and calendar week when the store started participating in Promo2
- **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [8]:
#path = 'data/train.csv'
#repo = 'https://github.com/emtinanseo/Sales-Future-Forecast.git'
#version = 'v0'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)

In [9]:
# import the train data: train.csv - historical data including Sales

data_train = pd.read_csv('../data/train.csv')
data_train.head(4)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1


In [10]:
#path = 'data/test.csv'
#repo = 'https://github.com/emtinanseo/Sales-Future-Forecast.git'
#version = 'v0'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)

In [11]:
# import the test data: test.csv - historical data excluding Sales

data_test = pd.read_csv('../data/test.csv')
data_test.head(4)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.00,1,0,0
1,2,3,4,2015-09-17,1.00,1,0,0
2,3,7,4,2015-09-17,1.00,1,0,0
3,4,8,4,2015-09-17,1.00,1,0,0


In [12]:
#path = 'data/store.csv'
#repo = 'https://github.com/emtinanseo/Sales-Future-Forecast.git'
#version = 'v0'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)

In [13]:
# import the store data: store.csv - supplemental information about the stores

data_store = pd.read_csv('../data/store.csv')
data_store.head(4)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,"1,270.00",9.00,"2,008.00",0,NaN,NaN,NaN
1,2,a,a,570.00,11.00,"2,007.00",1,13.00,"2,010.00","Jan,Apr,Jul,Oct"
2,3,a,a,"14,130.00",12.00,"2,006.00",1,14.00,"2,011.00","Jan,Apr,Jul,Oct"
3,4,c,c,620.00,9.00,"2,009.00",0,NaN,NaN,NaN


# Merge The Data

We merge both data_train and data_test DataFrames with data_store DataFrame, on the column 'Store' which is store id.

In [14]:
train = data_train.merge(data_store, left_on = "Store", right_on = "Store", how = 'left')

print(data_train.shape, data_store.shape, train.shape)

(1017209, 9) (1115, 10) (1017209, 18)


In [15]:
test = data_test.merge(data_store, left_on = "Store", right_on = "Store", how = 'left')

print(data_test.shape, data_store.shape, test.shape)

(41088, 8) (1115, 10) (41088, 17)


# Exploring Train and Test Data

## Size

In [16]:
print("train data has {} rows and {} columns".format(train.shape[0],train.shape[1]))

train data has 1017209 rows and 18 columns


In [17]:
print("test data has {} rows and {} columns".format(test.shape[0],test.shape[1]))

test data has 41088 rows and 17 columns


In [18]:
total = train.shape[0] + test.shape[0]

print("of the total data, {:.2f}% is train and {:.2f}% is test".format(train.shape[0]*100/total,
                                                                      test.shape[0]*100/total))

of the total data, 96.12% is train and 3.88% is test


# Change Data to Time Series

In [19]:
train['Date'] = pd.to_datetime(train.Date)

In [20]:
test['Date'] = pd.to_datetime(test.Date)

In [21]:
train['Date'].describe()

count                 1017209
unique                    942
top       2015-07-31 00:00:00
freq                     1115
first     2013-01-01 00:00:00
last      2015-07-31 00:00:00
Name: Date, dtype: object

In [22]:
test['Date'].describe()

count                   41088
unique                     48
top       2015-09-17 00:00:00
freq                      856
first     2015-08-01 00:00:00
last      2015-09-17 00:00:00
Name: Date, dtype: object

Train data covers period from 2013-01-01 to 2015-07-31 (2 years and a half), while test data covers period from 2015-08-01 to 2015-09-17 (1 month and a half)

In [23]:
# Set the date as index
train.set_index('Date', inplace= True)
test.set_index('Date', inplace= True)

# Summary

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2015-07-31 to 2013-01-01
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Sales                      1017209 non-null  int64  
 3   Customers                  1017209 non-null  int64  
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  object 
 7   SchoolHoliday              1017209 non-null  int64  
 8   StoreType                  1017209 non-null  object 
 9   Assortment                 1017209 non-null  object 
 10  CompetitionDistance        1014567 non-null  float64
 11  CompetitionOpenSinceMonth  693861 non-null   float64
 12  CompetitionOpenSinceYear   693861 non-null   float64
 1

In [25]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 41088 entries, 2015-09-17 to 2015-08-01
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Id                         41088 non-null  int64  
 1   Store                      41088 non-null  int64  
 2   DayOfWeek                  41088 non-null  int64  
 3   Open                       41077 non-null  float64
 4   Promo                      41088 non-null  int64  
 5   StateHoliday               41088 non-null  object 
 6   SchoolHoliday              41088 non-null  int64  
 7   StoreType                  41088 non-null  object 
 8   Assortment                 41088 non-null  object 
 9   CompetitionDistance        40992 non-null  float64
 10  CompetitionOpenSinceMonth  25872 non-null  float64
 11  CompetitionOpenSinceYear   25872 non-null  float64
 12  Promo2                     41088 non-null  int64  
 13  Promo2SinceWeek            23

In [26]:
# First let's classify the columns into

numeric_cols = ['Sales', 'Customers']
id_cols = ['Store', 'Id']
date_cols = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']
object_cols = ['DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday','StoreType', 'Assortment','Promo2',
               'PromoInterval']

In [27]:
train[numeric_cols].describe()

,Sales,Customers
count,"1,017,209.00","1,017,209.00"
mean,"5,773.82",633.15
std,"3,849.93",464.41
min,0.00,0.00
25%,"3,727.00",405.00
50%,"5,744.00",609.00
75%,"7,856.00",837.00
max,"41,551.00","7,388.00"


In [32]:
eda.categorical_describe(train, object_cols)

,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,Promo2,PromoInterval
count %,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,50.06
unique,7,2,2,5,2,4,3,2,3
top,4,1,0,0,0,a,a,1,"Jan,Apr,Jul,Oct"
freq %,14.34,83.01,61.85,84.06,82.14,54.23,52.84,50.06,28.82


In [33]:
eda.categorical_describe(test, object_cols)

,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,Promo2,PromoInterval
count %,100.00,99.97,100.00,100.00,100.00,100.00,100.00,100.00,58.06
unique,7,2.00,2,2,2,4,3,2,3
top,1,1.00,0,0,0,a,c,1,"Jan,Apr,Jul,Oct"
freq %,14.58,85.41,60.42,99.56,55.65,53.86,49.53,58.06,33.53


These tables are not enough to see full distribution of all variables, but we can compare binary variable (which take 0 or 1 as values). We see that:
- 'Open', 'Promo' and 'Promo2' columns have similar distributions in train and test. The corresponding train|test freq% for
    - 'Open' = 1 is 83%|85%
    - 'Promo' = 0 is 61%|60%
    - 'Promo2' = 1 is 50%|58%

# Missing Data _ First Look

In [25]:
eda.missing_values_table(train)

Your selected dataframe has 18 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Promo2SinceWeek,508031,49.90,float64
Promo2SinceYear,508031,49.90,float64
PromoInterval,508031,49.90,object
CompetitionOpenSinceMonth,323348,31.80,float64
CompetitionOpenSinceYear,323348,31.80,float64
CompetitionDistance,2642,0.30,float64


In [26]:
eda.missing_values_table(test)

Your selected dataframe has 17 columns.
There are 7 columns that have missing values.


,Missing Values,% of Total Values,Dtype
Promo2SinceWeek,17232,41.90,float64
Promo2SinceYear,17232,41.90,float64
PromoInterval,17232,41.90,object
CompetitionOpenSinceMonth,15216,37.00,float64
CompetitionOpenSinceYear,15216,37.00,float64
CompetitionDistance,96,0.20,float64
Open,11,0.00,float64


In [27]:
# how does the missing data look like
test[test['Open'].isna()]

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
479,480,622,4,2015-09-17,NaN,1,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
1335,1336,622,3,2015-09-16,NaN,1,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
2191,2192,622,2,2015-09-15,NaN,1,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
3047,3048,622,1,2015-09-14,NaN,1,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
4759,4760,622,6,2015-09-12,NaN,0,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
5615,5616,622,5,2015-09-11,NaN,0,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
6471,6472,622,4,2015-09-10,NaN,0,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
7327,7328,622,3,2015-09-09,NaN,0,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
8183,8184,622,2,2015-09-08,NaN,0,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
9039,9040,622,1,2015-09-07,NaN,0,0,0,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN


We cannot drop these rows, even though they have missing data, because the data is a time series. However we note that missing values in "Open" are from 1 Store 622

# Change Data to Time Series

In [32]:
train['Date'] = pd.to_datetime(train.Date)

In [33]:
test['Date'] = pd.to_datetime(test.Date)

In [38]:
train.set_index('Date', inplace= True)
test.set_index('Date', inplace= True)

# Explore

In [ ]:
test